In [37]:
#import libraries
import xgboost as xg
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib as plt
from scipy.stats import norm
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold, KFold
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization
from sklearn.metrics import r2_score
import data_ML as ML
import pickle

np.set_printoptions(suppress=True)

In [4]:
df = ML.getdata(False)


In [5]:
def modifydata(df):
    data = []
    df_calls = df[df.optionType == 1].drop(columns='optionType')
    df_american_calls = df_calls[df_calls.method == 1].drop(columns='method')
    # df_american_calls = df_american_calls.drop(columns='index')
    df_american_calls = df_american_calls[df_american_calls.lastPrice > 5]
    # df_american_calls = df_american_calls.drop(columns='index')
    y = np.asarray(df_american_calls[['lastPrice']])
    df_american_calls = df_american_calls.drop(columns='lastPrice')

    for line in range(len(df_american_calls)):
        data_tmp = df_american_calls.iloc[line]
        data.append(data_tmp)
    data = np.asarray(data)

    return data, y

# df_train = ML.getdata(True)


X, y = modifydata(df)
# df_train, y_train, full_data2 = modifydata(df_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1236, 5) (413, 5) (1236, 1) (413, 1)


In [14]:

# df_train, y_train, full_data2 = modifydata(df_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=3)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1236, 5) (413, 5) (1236, 1) (413, 1)


In [ ]:
#ANN with four layers, 10 neurons each
#activation function: ReLU

model_american_calls = Sequential()
model_american_calls.add(Dense(512, input_dim=5, activation='relu'))
model_american_calls.add(Dropout(0.2))
model_american_calls.add(Dense(256, activation='relu'))
model_american_calls.add(Dropout(0.2))
model_american_calls.add(Dense(256, activation='relu'))
model_american_calls.add(Dropout(0.2))
model_american_calls.add(Dense(64, activation='relu'))
model_american_calls.add(Dense(1))

# model = Sequential()
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(X.shape[1], 1)))
# model.add(MaxPooling1D(pool_size=2, strides=3))
# model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
# model.add(MaxPooling1D(pool_size=2, strides=3))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(1))

# model = Sequential()
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu', padding='same', input_shape=(X.shape[1], 1)))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2, strides=3))
# model.add(Conv1D(filters=16, kernel_size=3, activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2, strides=3))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.2))
# model.add(Dense(1))

from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=15, verbose=1)

# from keras.optimizers import SGD, RMSprop

# sgd = SGD(lr=0.01, momentum=0.9)
# rmsprop = RMSprop(lr=0.001)

# model.compile(optimizer=rmsprop, loss='mse')
model_american_calls.compile(optimizer='adam', loss='mse')

history = model_american_calls.fit(df_train, y_train, epochs=20, validation_split=0.20)



Epoch 1/20
319/319 [==============================] - 1s 2ms/step - loss: 512.6419 - val_loss: 511.5859
Epoch 2/20
319/319 [==============================] - 1s 2ms/step - loss: 264.5244 - val_loss: 423.4411
Epoch 3/20
319/319 [==============================] - 1s 2ms/step - loss: 247.8238 - val_loss: 855.0541
Epoch 4/20
319/319 [==============================] - 1s 2ms/step - loss: 227.5238 - val_loss: 494.8990
Epoch 5/20
319/319 [==============================] - 1s 2ms/step - loss: 200.9546 - val_loss: 334.7143
Epoch 6/20
319/319 [==============================] - 1s 2ms/step - loss: 209.6776 - val_loss: 240.4751
Epoch 7/20
319/319 [==============================] - 1s 2ms/step - loss: 191.6341 - val_loss: 225.0697
Epoch 8/20
319/319 [==============================] - 1s 2ms/step - loss: 182.4523 - val_loss: 366.3501
Epoch 9/20
319/319 [==============================] - 1s 2ms/step - loss: 172.6037 - val_loss: 210.7865
Epoch 10/20
319/319 [==============================] - 1s 2ms/st

In [ ]:
#prediction
import pandas as pd
y_pred = model_american_calls.predict(df_test)


db = pd.DataFrame({'prediction':y_pred.tolist(),'real':y_test.tolist()})
error = (y_pred - y_test)/y_test
db = db.assign(error = error)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(rmse)
db.to_csv('./Data/MLPdata.csv')




  1/399 [..............................] - ETA: 4s

399/399 [==============================] - 0s 534us/step
6.08928660569831


In [ ]:
y_test.shape

(3185,)

In [11]:
DTR1 = DecisionTreeRegressor()
# cvs_DTR = cross_val_score(DTR, df_train, y_train, scoring='neg_root_mean_squared_error', cv=5)
# DTR_score = cvs_DTR.mean()
# print(DTR_score)

DTR1.fit(X_train, y_train)
DTR1_pred = DTR1.predict(X_test)
DTR1_rmse = np.sqrt(mean_squared_error(y_test, DTR1_pred))
DTR1_rmse

12.169540161739228

In [13]:
y_pred = pd.DataFrame(DTR1_pred, columns=['prediction'])
final_test = pd.DataFrame(y_test, columns=['real'])

final_df = pd.concat((final_test, y_pred), axis=1)
final_df

,real,prediction
0,157.90,156.20
1,310.63,303.02
2,72.11,94.65
3,145.60,113.50
4,87.38,85.45
...,...,...
408,150.70,150.10
409,140.26,145.39
410,73.95,70.09
411,71.90,73.23


In [23]:

XGBr = xg.XGBRegressor(learning_rate=0.1, n_estimators=900)
cvs_XGBr = cross_val_score(XGBr, X_train, y_train, scoring='neg_root_mean_squared_error', cv=KFold(n_splits=10))
XGBr_score = cvs_XGBr.mean()
print(XGBr_score)


-5.72053084070279


In [36]:
XGBr.fit(X_train, y_train)
XGBr_pred = XGBr.predict(X_test)
XGBr_rmse = np.sqrt(mean_squared_error(y_test, XGBr_pred))
XGBr_rmse
saved_model = pickle.dumps(XGBr)

NameError: name 'pickle' is not defined

In [34]:
def xgb_prediction(S_0, r, T, K , sigma):
    data = np.asarray([S_0, r, T, K, sigma])
    prediction = XGBr.predict(data)
    return prediction



In [35]:
print(xgb_prediction(125.7, 0.256, 21/365, 50, 0.3326))

XGBoostError: [16:23:41] /Users/runner/work/xgboost/xgboost/src/predictor/cpu_predictor.cc:719: Check failed: m->NumColumns() == model.learner_model_param->num_feature (1 vs. 5) : Number of columns in data must equal to trained model.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000016d4f0998 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x000000016d6f0e60 void xgboost::predictor::CPUPredictor::DispatchedInplacePredict<xgboost::data::ArrayAdapter, 64ul>(std::__1::any const&, std::__1::shared_ptr<xgboost::DMatrix>, xgboost::gbm::GBTreeModel const&, float, xgboost::PredictionCacheEntry*, unsigned int, unsigned int) const + 344
  [bt] (2) 3   libxgboost.dylib                    0x000000016d6e8068 xgboost::predictor::CPUPredictor::InplacePredict(std::__1::shared_ptr<xgboost::DMatrix>, xgboost::gbm::GBTreeModel const&, float, xgboost::PredictionCacheEntry*, unsigned int, unsigned int) const + 1572
  [bt] (3) 4   li